## Neglect this part. Set up variable and env

In [1]:
import sys
sys.path.append('/Users/joy.chen/insitroRepos/posh_internal/common_utils') ## Adding path to common utils
sys.path.append('/Users/joy.chen/insitroRepos/posh_internal/core')


%load_ext autoreload
%autoreload 2

In [2]:
import database_utils as db_utils
import image_utils as img_utils

from skimage import exposure,filters,measure,segmentation

In [4]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

plate_id = "PA16998"
output_bucket = 's3://insitro-microscopy-data/prod/pyxcell'
output_dir = os.path.join(output_bucket,plate_id)
output_dir

's3://insitro-microscopy-data/prod/pyxcell/PA16998'

In [6]:
ordered_measurement_corrected = ['8c000250-41ab-4a3a-a486-6e5e4958108b',
 'ed70a5a8-d57c-4bad-a7e5-48f9ac0eeb9b',
 'e3efea0b-b0b6-4b97-94e6-b0fa1b1cf810',
 '58ef1bd6-22e3-46aa-890b-2ab86e06ecd6',
 '34d43556-7135-45ee-b2f7-f54c3e1774d3',
 'b04300f5-9804-423b-a6ad-a614820f5309',
 'dbc4abba-bfb5-4598-b822-0ad4f568c612',
 'd3ed20b3-5990-4d01-902d-eee87cc47676',
 '5ce71785-2895-468f-a49e-14a61a21fa0b',
 '0163135a-0f15-4893-b8b3-20d5b99b1cea',
 '1390bf25-01c6-4a8b-86e5-112f03488123',
 '7fbffcc0-ac76-483d-8c95-b16c0d71119a',
 'a803cd01-b3a4-445f-b97a-2f1a25023300',
 'd91ff1b3-0aeb-4588-9a0c-69b16cb4017d',
 'fac86e82-542e-4687-82b8-9902b735e3b8',
 'aa9a1fcd-1fac-4a64-80d5-56c7fd5ae79c',
 '6924fca1-51ea-41d3-9750-935a58bafcdf']

## ---------------- Read from here ----------------------  

In [15]:
# get "indexed_cells" df
path = os.path.join(os.path.join(os.path.join(output_dir,'annotated_coordinates/'),ordered_measurement_corrected[0]),
                    'indexed_cell_coordinates/coordinates.pq')
indexed_cells = db_utils.load_df(path)

In [12]:
# get "posh_barcodes_uncorrected" from import df
path = os.path.join(os.path.join(os.path.join(output_dir,'assign_barcodes_to_cells/nearest_label/'),ordered_measurement_corrected[0]),
                    'posh_barcodes_assigned_to_nuclei_v2_fcn.pq')
posh_barcodes_uncorrected = db_utils.load_df(path)

print("Number of cells from raw data:", posh_barcodes_uncorrected["cell_id"].nunique())
print("Number of unique barcode from raw data:", posh_barcodes_uncorrected["barcode"].nunique())

Number of cells from raw data: 2088484
Number of unique barcode from raw data: 8580073


In [14]:
#drop duplicate barcode in a cell
all_cells_with_unique_uncorrected_barcode = posh_barcodes_uncorrected.drop_duplicates(['cell_id','barcode'])

print("Number of cells from this step:", all_cells_with_unique_uncorrected_barcode["cell_id"].nunique())
print("Number of unique barcode from this step:", all_cells_with_unique_uncorrected_barcode["barcode"].nunique())

Number of cells from this step: 2088484
Number of unique barcode from this step: 8580073


Removing cells that have been assigned a barcode but not tiled.

In [17]:
cells_ids_iss = set(all_cells_with_unique_uncorrected_barcode['cell_id']) 
indexed_cell_ids = set(indexed_cells['ID']) 

difference_between_indexed_and_barcoded_cells = cells_ids_iss.difference(indexed_cell_ids) 
cell_ids_iss_not_indexed = all_cells_with_unique_uncorrected_barcode[all_cells_with_unique_uncorrected_barcode['cell_id'].isin(list(difference_between_indexed_and_barcoded_cells))]
#df shows the cells are barcoded but not indexed. But should be noted that this df includes duplicate cells


print('Fraction of sequenced cells not indexed :', cell_ids_iss_not_indexed["cell_id"].nunique()/len(cells_ids_iss))


#Removing cells that have been assigned a barcode but not tiled
all_cells_with_unique_uncorrected_barcode_remove_non_index_cell =  all_cells_with_unique_uncorrected_barcode[~all_cells_with_unique_uncorrected_barcode['cell_id'].isin(list(difference_between_indexed_and_barcoded_cells))]

print("Number of unique cells which are barcoded but not indexed:", cell_ids_iss_not_indexed["cell_id"].nunique())
print("Number of unique barcodes from cells which are barcoded but not indexed:", cell_ids_iss_not_indexed["barcode"].nunique())

print("Number of unique cells after removing the cells that are barcodedbut not indexed:", all_cells_with_unique_uncorrected_barcode_remove_non_index_cell["cell_id"].nunique())
print("Number of unique barcodes after removing the cells that are barcodedbut not indexed:",all_cells_with_unique_uncorrected_barcode_remove_non_index_cell["barcode"].nunique())



Fraction of sequenced cells not indexed : 0.06627821903351905
Number of unique cells which are barcoded but not indexed: 138421
Number of unique barcodes from cells which are barcoded but not indexed: 514979
Number of unique cells after removing the cells that are barcodedbut not indexed: 1950063
Number of unique barcodes after removing the cells that are barcodedbut not indexed: 8213310
